In [31]:
import pandas as pd

file_path = 'data_clean/final_data_CD.csv'
data = pd.read_csv(file_path)

In [32]:
# function to classify debt holders + officials
def classify_debt_holder(row):
    name = str(row['Full Search Name']).lower()
    public_official_keywords = ['treasurer', 'public official', 'government', 'secretary', 'comptroller']

    if any(keyword in name for keyword in public_official_keywords):
        return 'public', row['Full Search Name']
    else:
        return 'private', None
    
data['debt_holder_type'], data['official'] = zip(*data.apply(classify_debt_holder, axis=1)) # apply function to dataset

In [33]:
# combine debt
debt_by_state = data.groupby(['Group State', 'debt_holder_type'])['final_total'].sum().unstack(fill_value=0)

# make sure public and private are columns
debt_by_state = debt_by_state.reindex(columns=['public', 'private'], fill_value=0)

debt_by_state.reset_index(inplace=True)

In [36]:
# rename columns so its easier to read
debt_by_state.columns = ['State', 'Public Debt', 'Private Debt']
debt_by_state['Public Debt'] = 0 # set public debt to zero because we don't have data for that

# make private debt readable
debt_by_state['Private Debt'] = debt_by_state['Private Debt'].apply(lambda x: f"{x:,.2f}")

In [37]:
state_names = {
    'CT': 'Connecticut', 'DE': 'Delaware', 'GA': 'Georgia', 'MA': 'Massachusetts', 'MD': 'Maryland',
    'NC': 'North Carolina', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NY': 'New York', 'PA': 'Pennsylvania',
    'RI': 'Rhode Island', 'SC': 'South Carolina', 'VA': 'Virginia', 'VT': 'Vermont'
}

# print final table w state column as a row
debt_by_state.set_index('State', inplace=True)
print(debt_by_state)

       Public Debt  Private Debt
State                           
CT               0  1,516,750.76
DE               0     16,903.78
GA               0     18,446.75
MA               0    163,989.20
MD               0  1,555,678.22
NC               0     53,942.82
NH               0    265,757.19
NJ               0     43,738.58
NY               0  1,866,244.90
PA               0  3,378,843.37
RI               0    844,056.80
SC               0  1,498,912.85
VA               0     32,357.20
VT               0      5,994.95
